In [295]:
# Prereq libraries
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pb
import random
from nltk import word_tokenize

# Possibly useful TensorFlow and Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM
from keras import layers
import tensorflow as tf

# Read in dataset from csv files
dataset = pb.read_csv('min_75_max5k_as_ids.csv', names = ["tags", "desc"])

In [296]:
from functs import npFloatArray

########## TEST VARIABLES

test_set_percentage = 0.75

#########################

# load features and labels based on those present in the CSV files
features = dataset.copy()

# convert features into numpy array
features = np.array(features)

# shape and extract total num
totalNum = np.shape(np.array(dataset["desc"]))[0]
trainNum = int(totalNum*test_set_percentage)
testNum = totalNum - trainNum

# create labels and descriptions separately
descs = dataset["desc"][:]
labels = dataset["tags"][:]

# Initialize test and train sets
trainSetX = []
trainSetY = []

testSetX = []
testSetY = []

# This whole next section is for data loading into numpy arrays, code is based on Assignment 1 util.py load_data() function
training_samples = []
random.seed(0)

# Iterate over dataset and add numpy arrays as elements into trainSetX (the descriptions) and trainSetY (the labels) 1 by 1
# Note: For some reason, certain cells (very few) store the data as floats and not strings, these were filtered out
for i in range(trainNum):
        found = False
        while not found:
            sample_index = random.randint(1, totalNum - 1)
            if sample_index not in training_samples:
                training_samples.append(sample_index)
                found = True
                if type(descs[sample_index]) == str and type(labels[sample_index]) == str:
                    tmp = list(descs[sample_index].split(", "))
                    trainSetX.append(np.array(tmp, float))
                    trainSetY.append(np.array(labels[sample_index].split(", ")))

# Create test set in a similar fashion to the train set    
test_index = 0
for i in range(1, totalNum):
    if i not in training_samples:
        if type(descs[i]) == str and type(labels[i]) == str:
            testSetX.append(np.array(list(descs[i].split(", "))))
            testSetY.append(np.array(list(labels[i].split(", "))))
            test_index += 1


# Convert sets into numpy arrays for tensor conversion
train_x = npFloatArray(np.array(trainSetX, dtype = object))
train_y = npFloatArray(np.array(trainSetY, dtype = object))
test_x = npFloatArray(np.array(testSetX, dtype = object))
test_y = npFloatArray(np.array(testSetY, dtype = object))


# Latest attempt at tensor conversion for model fit function
# This part will be fixed soon
train_x = np.array([np.array(val) for val in train_x])
train_y = np.array([np.array(val) for val in train_y])
train_x = tf.cast(train_x , dtype=tf.float32)
train_y = tf.cast(train_y , dtype=tf.float32)

# train_x sanity checks for shape and datatype
print(train_x)
print(type(train_x))

c:\Users\jailm\Desktop\School\2022-2023 WINTER\EECS 4404\Project_RNN\functs.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
C:\Users\jailm\AppData\Local\Temp\ipykernel_12612\4169642353.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_x = np.array([np.array(val) for val in train_x])
C:\Users\jailm\AppData\Local\Temp\ipykernel_12612\4169642353.py:69: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
# RNN model implementation using Keras Library

# Create sequential model object
model = Sequential()

# First hidden layer using LSTM with window size 64, train_x shape, & RelU activation [128 nodes]
model.add(LSTM(128, input_shape = (64, train_x.shape[0]), activation = 'relu', return_sequences = True))
model.add(Dropout(0.2))

# Second  hidden layer using LSTM with RelU activation [128 nodes]
model.add(LSTM(128, activation = 'relu'))
model.add(Dropout(0.2))

# Third hidden layer using fully conncted nodes with RelU activation [32 nodes]
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))

# Final output layer using softmax activation [10 nodes]
model.add(Dense(10, activation = 'softmax'))

# optimizer used for compilation, 'Adam' refers to gradient descent method that "adapts" using first and second order moments
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3, decay = 1e-5)

# Final compilation
model.compile(loss = 'sparse_ctegorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

# Model Fitting process, this is the last step
# Note: Currently having issues with Tensor conversion causing this step to not accept training data
model.fit(train_x, train_y, epochs = 3, validation_data = (test_x, test_y))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).